# Advanced Deep-Live-Cam - Face Swap with Mouth Mask
Complete face swap solution with mouth mask option for images and videos

In [ ]:
# Install system dependencies
!apt update -qq
!apt install -y ffmpeg

# Clone repository
!git clone https://github.com/Mayank-kanojiya/liveswap.git
%cd liveswap

In [ ]:
# Install Python dependencies
!pip install -q gradio opencv-python insightface onnxruntime-gpu torch torchvision
!pip install -q customtkinter pillow psutil opennsfw2 protobuf cv2-enumerate-cameras
!pip install -q git+https://github.com/xinntao/BasicSR.git@master
!pip install -q git+https://github.com/TencentARC/GFPGAN.git@master

In [ ]:
# Download required models
import os
os.makedirs('models', exist_ok=True)

!wget -q -O models/GFPGANv1.4.pth https://huggingface.co/hacksider/deep-live-cam/resolve/main/GFPGANv1.4.pth
!wget -q -O models/inswapper_128.onnx https://huggingface.co/hacksider/deep-live-cam/resolve/main/inswapper_128_fp16.onnx

In [ ]:
# Advanced Face Swap with Options
import gradio as gr
import cv2
import tempfile
import sys
import os
import shutil
sys.path.append('.')

# Initialize modules
import modules.globals
modules.globals.headless = True
modules.globals.execution_providers = ['CPUExecutionProvider']
modules.globals.frame_processors = ['face_swapper']
modules.globals.many_faces = False
modules.globals.mouth_mask = False

from modules.processors.frame.face_swapper import get_face_swapper, process_frame
from modules.face_analyser import get_one_face
from modules.utilities import is_video, extract_frames, create_video, restore_audio, get_temp_frame_paths, create_temp, clean_temp

# Initialize face swapper
face_swapper = get_face_swapper()

def process_media(source_image, target_file, mouth_mask, many_faces):
    if source_image is None or target_file is None:
        return None, "Please upload both source image and target media"
    
    try:
        # Set global options
        modules.globals.mouth_mask = mouth_mask
        modules.globals.many_faces = many_faces
        modules.globals.source_path = source_image
        
        # Read source image
        source_img = cv2.imread(source_image)
        if source_img is None:
            return None, "Could not read source image"
        
        # Get source face
        source_face = get_one_face(source_img)
        if source_face is None:
            return None, "No face detected in source image"
        
        # Check if target is video or image
        if is_video(target_file.name):
            # Video processing
            result_path = tempfile.mktemp(suffix='.mp4')
            
            # Create temp directory for frames
            create_temp(target_file.name)
            extract_frames(target_file.name)
            
            # Process each frame
            temp_frame_paths = get_temp_frame_paths(target_file.name)
            
            for frame_path in temp_frame_paths:
                frame = cv2.imread(frame_path)
                result_frame = process_frame(source_face, frame)
                cv2.imwrite(frame_path, result_frame)
            
            # Create output video
            create_video(target_file.name)
            restore_audio(target_file.name, result_path)
            
            # Clean up
            clean_temp(target_file.name)
            
            return result_path, "Video face swap completed successfully!"
            
        else:
            # Image processing
            target_img = cv2.imread(target_file.name)
            if target_img is None:
                return None, "Could not read target image"
            
            # Process frame
            result_frame = process_frame(source_face, target_img)
            if result_frame is None:
                return None, "Face swap failed - no face detected in target image"
            
            # Save result
            result_path = tempfile.mktemp(suffix='.jpg')
            cv2.imwrite(result_path, result_frame)
            
            return result_path, "Image face swap completed successfully!"
        
    except Exception as e:
        return None, f"Error: {str(e)}"

# Create advanced interface
with gr.Blocks(title="Advanced Deep-Live-Cam") as demo:
    gr.Markdown("# Advanced Deep-Live-Cam - Face Swap with Options")
    gr.Markdown("Upload source face image and target image/video. Enable options as needed.")
    
    with gr.Row():
        with gr.Column():
            source_image = gr.Image(label="Source Face Image", type="filepath")
            target_media = gr.File(label="Target Image/Video")
            
            with gr.Row():
                mouth_mask = gr.Checkbox(label="Mouth Mask (preserve original mouth)", value=False)
                many_faces = gr.Checkbox(label="Process All Faces", value=False)
            
            process_btn = gr.Button("Process Face Swap", variant="primary")
            
        with gr.Column():
            result_media = gr.File(label="Result")
            status_text = gr.Textbox(label="Status", interactive=False)
    
    process_btn.click(
        fn=process_media,
        inputs=[source_image, target_media, mouth_mask, many_faces],
        outputs=[result_media, status_text]
    )

demo.launch(share=True, debug=False)